In [ ]:
#coding=utf-8
import xml.dom.minidom
import re
import os
import sys

jira_issues = ['ace', 'activemq', 'aurora', 'beam', 'cassandra', 'couchdb', 'hbase', 'hive', 'incubator-systemml', 'maven', 'spark', 'zookeeper']

In [ ]:
def remove(data):
    data = re.sub(r'</?p>', "", data)
    data = re.sub(r'</?tt>', "", data)
    data = re.sub(r'<br/>', "", data)
    data = re.sub(r'\<a.*?\>', "", data)
    data = re.sub(r'</a>', "", data)
    data = re.sub(r'\<div.*?\>', "", data)
    data = re.sub(r'\</div\>', "", data)
    data = re.sub(r'\<pre.*?\>', "", data)
    data = re.sub(r'\</pre\>', "", data)
    data = re.sub(r'\<span.*?\>', "", data)
    data = re.sub(r'\</span\>', "", data)
    data = re.sub(r'\<ul.*?\>', "", data)
    data = re.sub(r'</ul\>', "", data)
    data = re.sub(r'\<table.*?\>', "", data)
    data = re.sub(r'\</table\>', "", data)
    data = re.sub(r'\<td.*?\>', "", data)
    data = re.sub(r'\</td\>', "", data)
    data = re.sub(r'\<th.*?\>', "", data)
    data = re.sub(r'\</th\>', "", data)
    data = re.sub(r'\</?del\>', "", data)
    data = re.sub(r'\</?em\>', "", data)
    data = re.sub(r'\</?h3\>', "", data)
    data = re.sub(r'\</?li\>', "", data)
    data = re.sub(r'</?ol>', "", data)
    data = re.sub(r'</?tr>', "", data)
    data = re.sub(r'</?tbody>', "", data)
    data = re.sub(r'\<img.*?\>', "", data)
    data = re.sub(r'\n', " ", data)
    data = re.sub(r'\&gt\;', ">", data)
    data = re.sub(r'\&lt\;', "<", data)
    data = re.sub(r'\&\#91\;', "[", data)
    data = re.sub(r'\&\#93\;', "]", data)
    data = re.sub(r'\&\#8211\;', "-", data)
    data = re.sub(r'\&amp\;', "&", data)
    data = re.sub(r'\<200c\>', "", data)
    data = re.sub(r'\<200b\>', "", data)
    return data

In [ ]:
def readInfoFromXML(parseFile):
    """return dataset, a list of dict data points"""
    dp = {}
    title = parseFile.getElementsByTagName('title')
    data = title[1].firstChild.data
    data = re.sub(r'\[.*?\]\s', "", data)
    data = remove(data)
    dp['title'] = data

    description = parseFile.getElementsByTagName('description')
    des_str = ''
    for i, des in enumerate(description):
        if i != 0 and des.firstChild != None:
            data = remove(des.firstChild.data)
            des_str = des_str + data
    dp['description'] = des_str.encode('utf-8')
    
    comments = parseFile.getElementsByTagName('comment')
    comment_str = ''
    for i, com in enumerate(comments):
        if com.firstChild:
            data = remove(com.firstChild.data)
            comment_str = comment_str + data
    dp['comment'] = comment_str.encode('utf-8')  
    
    type = parseFile.getElementsByTagName('type')
    dp['type'] = type[0].firstChild.data.encode('utf-8')
        
    priority = parseFile.getElementsByTagName('priority')
    for i, pri in enumerate(priority):
        dp['priority'] = pri.firstChild.data.encode('utf-8')
        
    return dp

In [ ]:
def getInfoFromXML(fromDir, path):
    fromFile = os.path.join('%s%s' % (fromDir, path))
    dom = xml.dom.minidom.parse(fromFile)
    parseFile = dom.documentElement
    return readInfoFromXML(parseFile)

In [ ]:
def getUsefulInfo(filepath):
    dataset = []
    pathDir =  os.listdir(filepath)
    invalidName = "invalid"
    numFiles = 0
    for allDir in pathDir:
        if 'GitHub' not in allDir:
            if invalidName in allDir:
                continue
            dp = getInfoFromXML(filepath, allDir)
            dataset.append(dp)
            numFiles += 1
    return dataset

datasets = {}
total = 0
for ji in jira_issues:
    filePath = './data-jira/' + ji + '-issues/'
    datasets[ji] = getUsefulInfo(filePath)
    total += len(datasets[ji])
    print('------ %s -------' % ji)
    print('Number of patches:', len(datasets[ji]))
    print('Percentage of bugs: %.2f' % (1.0 * sum([1 for dp in datasets[ji] if dp['type'] == 'Bug']) / len(datasets[ji])))
    
print('Number of patches in total:', total)

In [ ]:
for ji in jira_issues:
    for dp in datasets[ji]:
        if 'description' not in dp:
            datasets[ji].remove(dp)
        if 'comment' not in dp:
            datasets[ji].remove(dp)
print('Number of patches left: ', sum([len(datasets[ji]) for ji in jira_issues]))

In [ ]:
import pickle
pickle.dump(datasets, open('data-jira/jira-issues.pickle', 'wb'), True)